# Bayesian optimization with `skopt`, `skorch` and `comet`


In [0]:
%%writefile .comet.config
[comet]
api_key=<YOUR KEY>
logging_file = /tmp/comet.log
logging_file_level = info

In [0]:
# sklearn version fixed to avoid known skopt issue
!pip install scikit-optimize scikit-learn==0.20.3 skorch comet_ml

In [0]:
import numpy as np
from comet_ml import Experiment

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from skopt import BayesSearchCV


Let's optimise the neural net from the previous notebook via `BayesSearchCV` 


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
torch.manual_seed(0);

In [0]:
from sklearn.datasets import make_classification
X, y = make_classification(1000, 20, n_informative=10, n_classes=2, random_state=0)
X = X.astype(np.float32)

### Neural net module

In [0]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

### Callback mechanism

In [0]:
from skorch.callbacks import Callback
class ExperimentCallback(Callback):
    def __init__(self, name, key, **kwargs):
      self.name = name
      self.key = key
      self.experiment = None

    def initialize(self):
      self.experiment = Experiment(api_key=self.key,
                      project_name=self.name)

    def on_train_begin(self, net, **kwargs):
      pass
        
    def on_train_end(self, net, **kwargs):
      train_loss = net.history[-1, 'train_loss']
      valid_loss = net.history[-1, 'valid_loss']
      self.experiment.log_metrics({'train_loss': train_loss,
                                   'valid_loss': valid_loss})
      self.experiment.end()


In [0]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device='cuda',  # comment this to train with CPU
    optimizer__momentum=0.9,
    verbose=0,
    callbacks = [ExperimentCallback("comet demo5", YOUR_API_KEY)]
)

In [0]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20, 30], # range from 10 to 50
    'module__dropout': [0.1, 0.3], # range from 0.1 to 0.3
    'optimizer__nesterov': [False, True],
}

In [0]:
bs = BayesSearchCV(net, params, refit=False, cv=2, scoring='accuracy', 
                  verbose=0, n_jobs=1, n_iter=3, return_train_score=True)

In [0]:
bs.fit(X, y);

## Task
1. examine project at [comet.ml](https://www.comet.ml) 

2. change search space, number of iterations, save and run to another project